In [1]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
emb = torch.rand(32,256,256) # bs,256, 256
emb.shape

32

In [21]:
emb = torch.rand(32,256,256)
class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        #channel wise
        self.fc1 = nn.Linear(in_features=256, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=256)
        self.relu = nn.LeakyReLU(negative_slope=0.3)
        self.sigmoid = nn.Sigmoid()
        
        #feature wise
        
    def channel_wise(self,emb):
        emb = F.adaptive_avg_pool1d(emb, output_size=1) #bs,256,1
        emb= emb.view(emb.shape[0], 1,256) #bs,1,256
        emb = self.fc1(emb) #bs,1,64
        emb= emb.view(emb.shape[0], 64,1) #,bs,64,1
        emb= self.relu(emb) #,bs,64,1
        
        emb= emb.view(emb.shape[0], 1,64) #bs,1,64
        emb= self.fc2(emb) #bs,1,256
        emb= emb.view(emb.shape[0], 256,1) #bs,256,1
        emb = self.sigmoid(emb)
        return emb 
    def feature_wise(self, emb):
        emb = torch.transpose(emb,1,2) #bs,256,256
        emb = F.adaptive_avg_pool1d(emb, output_size=1) #bs,256,1
        emb= emb.view(emb.shape[0], 1,256) #bs,1,256
        emb = self.fc1(emb) #bs,1,64
        emb= emb.view(emb.shape[0], 64,1) #,bs,64,1
        emb= self.relu(emb) #,bs,64,1
        
        emb= emb.view(emb.shape[0], 1,64) #bs,1,64
        emb= self.fc2(emb) #bs,1,256
        emb= emb.view(emb.shape[0], 256,1) #bs,256,1
        emb = torch.transpose(emb,1,2) #bs,1,256
        emb = self.sigmoid(emb)
        return emb
attention = Attention()
emb1 = attention.channel_wise(emb)
emb2 = attention.feature_wise(emb)
emb = emb1*emb*emb2
emb.shape
    

torch.Size([32, 256, 256])